### Problema de la mochila (knapsack) (versión 0-1)

__Entradas:__ Límite de peso $W$, lista de pesos de los ítems $[w_1, \ldots, w_k]$, y lista de valores de los ítems $[v_1, \ldots, v_k]$.

__Salida:__ Para cada ítem, podemos elegir incluirlo en la mochila ($n_i = 1$) o dejarlo fuera ($n_i = 0$) de modo que
   1. El peso total sea menor o igual al límite de la mochila: $n_1 w_1 + \cdots + n_k w_k \leq W$. Nótese que cada $n_i \in \{0, 1\}$, dependiendo de si el ítem \# i se elige o no.
   2. Se maximice el valor de los bienes robados: $n_1 v_1 + \ldots + n_k v_k$ es máximo.

In [ ]:
# Importante: Ejecuta esta celda a continuación
W = 200  # Límite de peso es 200
pesos = [1, 5, 20, 35, 90]  # Estos son los pesos de los ítems individuales
valores = [15, 14.5, 19.2, 19.8, 195.2]  # Estos son los valores de los ítems individuales

#### 1. Identificar la subestructura óptima

Supongamos que el límite de peso actual es $W$ y ya hemos tomado decisiones para todos los ítems desde $1, \ldots, j-1$, donde $j \geq 1$. ¿Qué decisiones podemos tomar para el ítem \# $j$?

   1. Remover el ítem $j$: el límite de peso restante es $W - w_j$ y obtenemos un valor de $v_j$. El __problema restante__ es encontrar la mejor forma de remover con un límite de peso $W - w_j$ usando los ítems de $j+1, \ldots, n$.
   2. No remover el ítem $j$: el límite de peso restante sigue siendo $W$ y no se obtiene ningún valor, ya que no se retira el ítem $j$. El __problema restante__ es encontrar la mejor forma de retirar con un límite de peso $W$ usando los ítems de $j+1, \ldots, n$.

Podemos ver así que el problema tiene una subestructura óptima:
 - Podemos tomar las decisiones en _etapas_, en este caso un ítem a la vez.
 - Una vez tomada una decisión, el problema restante es también una instancia del problema original, pero con datos modificados.

#### 2. Recurrencia
$$\newcommand\msz{\text{maxValorRetirado}}$$
$$\msz(W, j) = \max\ \left\{ \begin{array}{ll}
v_j + \msz(W - w_j, j+1), & \leftarrow \ \text{remover el ítem } j \\ 
\msz(W, j+1) & \leftarrow \ \text{omitir el ítem } j \\ 
\end{array} \right.$$

Casos base:

  * $\msz(0, j) = 0$, para todo $j \in \{1, \ldots, n\}$. Esto maneja el caso cuando no queda capacidad de peso.
  * $\msz(W, j) = -\infty$ si $W < 0$, para todo $j \in \{1, \ldots, n\}$. Esto maneja el caso cuando se violan las restricciones de peso.
  * $\msz(W, n+1) = 0$, para todo $W \geq 0$. Esto maneja el caso cuando se han agotado los ítems para remover.


In [ ]:
def maxValorRetirado(W, j, pesos, valores):
    assert j >= 0 
    assert len(pesos) == len(valores)
    # pesos -- lista de pesos de los ítems
    # valores -- lista de valores de los ítems
    # W: límite de peso
    # j: número del ítem que estamos considerando

    # Primero, los casos base
    if W == 0:
        return 0
    if W < 0:  # Se han agregado más ítems a la mochila de lo que permite su capacidad
        return -float('inf')
    if j >= len(pesos):
        return 0
    # A continuación, se maneja la recurrencia.
    return max(
        valores[j] + maxValorRetirado(W - pesos[j], j+1, pesos, valores),  # remover el ítem j
        maxValorRetirado(W, j+1, pesos, valores)  # omitir el ítem j
    )

In [ ]:
maxValorRetirado(W, 0, pesos, valores)

In [ ]:
maxValorRetirado(20, 0, pesos, valores)

#### 3. Memoización

La memoización de la recurrencia $\msz$ la convertirá en una tabla.
 - La entrada de la tabla $T[(w, j)]$ representará el valor de $\msz(w,j)$ para el límite de peso $0 \leq w \leq W$ y $1 \leq j \leq n$.
 - Supondremos que $T[(0, *)] = 0$ y $T[(*, n+1)] = 0$, donde * denota un número arbitrario para ese argumento.
 - Si intentamos acceder a $T[(w, *)]$ para $w < 0$, asumiremos que su valor es $-\infty$.


### 4. Recuperar la solución

Almacenamos en una tabla separada $S[(0,0)], \ldots, S[(W,n)]$ qué opción nos proporciona el mejor valor:
  - $S[(w, j)] = +1$ significa que, para el límite de peso $w$, elegiremos incluir el ítem $j$.
  - $S[(w,j)] = 0$ significa que, para el límite de peso $w$, omitiremos el ítem $j$.

El objetivo será llenar primero las tablas $T$ y $S$ con las entradas del problema y luego recuperar la solución.

Recordemos nuevamente la recurrencia:
$$\msz(W, j) = \max\ \left\{ \begin{array}{ll}
v_j + \msz(W - w_j, j+1), & \leftarrow \ \text{remover el ítem } j \\ 
\msz(W, j+1) & \leftarrow \ \text{omitir el ítem } j \\ 
\end{array} \right.$$

Vemos que $\msz(w,j)$ requiere conocer $\msz(w', j+1)$ para $w' \leq w$.
 - Por lo tanto, la tabla debe llenarse con $w = 0, \ldots, W$ en orden ascendente y $j = n, \ldots, 1$ en orden descendente.

Esto es importante a tener en cuenta para nuestro algoritmo de memoización.


In [ ]:
def memoizedMaxValorRetirado(W, pesos, valores): 
    n = len(pesos)
    assert (len(valores) == n), 'La lista de pesos y valores debe tener el mismo tamaño'
    assert (W >= 0)
    if W == 0: 
        return 0, []  # Nada que remover y valor 0.
    
    # Inicializar la tabla de memoización como una lista de listas
    # Llenar todas las entradas con 0
    T = [[0 for j in range(n)] for w in range(W+1)]
    S = [[0 for j in range(n)] for w in range(W+1)]

    # Usaremos este método auxiliar para acceder a nuestra tabla de memoización.
    # Esto nos ahorrará código más adelante.
    def getTblEntry(w, j): 
        if w == 0: 
            return 0
        if w < 0: 
            return -float('inf')
        if j >= n:
            return 0
        return T[w][j]

    for w in range(1, W+1):  # w en orden ascendente de 1 a W.
        for j in range(n-1, -1, -1):  # Bucle en orden descendente de n-1 a 0.
            # Esto nos permite llenar T y S simultáneamente sin usar estructura if-then-else
            (T[w][j], S[w][j]) = max(
                (valores[j] + getTblEntry(w - pesos[j], j+1), 1), 
                (getTblEntry(w, j+1), 0)
            )
    itemsToSteal = [] 
    # Recuperar la solución
    weightOfKnapsack = W  
    for j in range(n): 
        if (S[weightOfKnapsack][j] == 1):
            itemsToSteal.append(j)
            weightOfKnapsack = weightOfKnapsack - pesos[j]
            print(f'Remover ítem {j}: Peso = {pesos[j]}, Valor = {valores[j]}')
    print(f'Peso total retirado: {W - weightOfKnapsack}, valor = {T[W][0]}')
    return (T[W][0], itemsToSteal)


In [ ]:
memoizedMaxValorRetirado(W, pesos, valores)

In [ ]:
memoizedMaxValorRetirado(20, pesos, valores)

In [ ]:
memoizedMaxValorRetirado(150, pesos, valores)

### Problema de la mochila con número ilimitado de ítems

Estudiaremos una versión del problema de la mochila en la que se puede elegir cada ítem un número ilimitado de veces.

__Entradas:__ Límite de peso $W$, lista de pesos de los ítems $[w_1, \ldots, w_k]$, y lista de valores de los ítems $[v_1, \ldots, v_k]$.

__Salida:__ Elegir cuántos de cada ítem tomar $[n_1, \ldots, n_k]$ de modo que
   1. El peso total sea menor o igual al límite de la mochila: $n_1 w_1 + \cdots + n_k w_k \leq W$.
   2. Se maximice el valor de los bienes removidos: $n_1 v_1 + \ldots + n_k v_k$ es máximo.

In [ ]:
W = 200
pesos = [1, 5, 20, 35, 90]
valores = [15, 14.5, 19.2, 19.8, 195.2]

#### 1. Identificar la subestructura óptima

Supongamos que el límite de peso actual es $W$. Comprometámonos a remover uno de los ítems disponibles y veamos qué queda por hacer.

   1. Supongamos que decidimos remover el ítem $j$.
   2. Ahora necesitamos resolver el mismo problema pero para un límite de peso $W - w_j$. Si se obtiene la solución para este subproblema, entonces la solución del problema original es tomar la solución para $W - w_j$ y agregarle el ítem $j$.

De esta forma, vemos que el problema tiene una subestructura óptima.

#### 2. Recurrencia

$$\text{maxRemovido}(W) = \max\ \left\{ \begin{array}{ll}
0 & \leftarrow \ \text{¡Elegir no remover nada y detenerse!}\\
v_1 + \text{maxRemovido}(W - w_1) & \leftarrow \ \text{Elegir una unidad del ítem } 1 \\
v_2 + \text{maxRemovido}(W - w_2) & \leftarrow \ \text{Elegir una unidad del ítem } 2 \\
\vdots & \\
v_k + \text{maxRemovido}(W - w_k) & \leftarrow \ \text{Elegir una unidad del ítem } k\\
\end{array} \right.$$

Caso Base:

  * $\text{maxRemovido}(0) = 0$
  * $\text{maxRemovido}(W) = -\infty$ si $W < 0$.


In [ ]:
def maxRemovido(W, pesos, valores):
    if W == 0:
        return 0
    if W < 0:
        return -float('inf')
    k = len(pesos)
    assert len(valores) == k
    opts = [ valores[i] + maxRemovido(W - pesos[i], pesos, valores) for i in range(k) ]
    return max(opts)

In [ ]:
print(maxRemovido(25, pesos, valores))
# ADVERTENCIA: Esto se ejecutará durante mucho tiempo.
#print(maxRemovido(W, pesos, valores))

#### 3. Memoización

La memoización es muy sencilla. Creamos una tabla $T[0], \ldots, T[W]$ para almacenar $\text{maxRemovido}(j)$ para $j$ que varía de $0$ a $W$.
El resto sigue la estructura de la recurrencia, teniendo cuidado de manejar por separado los valores negativos de peso.

#### 4. Recuperar la solución

Almacenamos en una tabla separada $S[0], \ldots, S[W]$ qué opción nos proporciona el mejor valor.

In [ ]:
def memo_maxRemovido(W, pesos, valores):
    # Inicializar las tablas
    T = [0] * (W+1)
    S = [-1] * (W+1)
    k = len(pesos)
    assert len(valores) == k
    for w in range(1, W+1):
        opts = [((valores[i] + T[w - pesos[i]]), i) for i in range(k) if w - pesos[i] >= 0]
        opts.append((-float('inf'), -1))  # En caso de que opts esté vacío en el paso anterior.
        T[w], S[w] = max(opts)
    # Esto finaliza el cálculo
    rem_item_ids= []
    weight_remaining = W
    while weight_remaining >= 0:
        item_id = S[weight_remaining]
        if item_id >= 0:
            rem_item_ids.append('Remover ítem ID %d: peso = %d, valor = %f' % (item_id, pesos[item_id], valores[item_id]))
            weight_remaining = weight_remaining - pesos[item_id]
        else:
            break
    return T[W], rem_item_ids

In [ ]:
print(memo_maxRemovido(25, pesos, valores))
print(memo_maxRemovido(W, pesos, valores))

#### Comentarios


El problema de la mochila es un problema clásico de optimización combinatoria en el que se tiene un conjunto de ítems, cada uno con un peso $ w_i $ y un valor $ v_i $, y se desea seleccionar una combinación de estos ítems de tal manera que:
1. La suma total de los pesos de los ítems seleccionados no exceda la capacidad $ W $ de la mochila.
2. La suma de los valores de los ítems seleccionados sea lo mayor posible.

Existen varias versiones del problema. En el ejemplo del código se trabajan dos variantes:

1. **Mochila 0-1**: Cada ítem puede ser tomado o no tomado (se representa con una decisión binaria: incluir o no incluir el ítem).  
2. **Mochila con ítems ilimitados**: Se permite tomar cada ítem tantas veces como se desee, siempre y cuando se cumpla la restricción de capacidad.

La versión 0-1 es la que se usa con más frecuencia para ilustrar la aplicación de la programación dinámica. El código muestra cómo se puede definir una función recursiva que, en cada llamada, evalúa si incluir o excluir el ítem actual, basándose en la subestructura óptima del problema.


Para la versión 0-1, el problema se puede formular recursivamente. La función $\text{maxValorRetirado}(W, j)$ representa el valor máximo que se puede obtener con un límite de peso $ W $ considerando únicamente los ítems a partir del índice $ j $. La recursión se basa en dos decisiones fundamentales para cada ítem $ j $:

1. **Incluir el ítem $ j $**: Si se decide tomar el ítem, se suma su valor $ v_j $ y se reduce la capacidad en $ w_j $, resolviendo el subproblema $\text{maxValorRetirado}(W - w_j, j+1)$.
2. **Omitir el ítem $ j $**: Si no se incluye el ítem, se conserva la capacidad $ W $ y se continúa con el siguiente ítem, es decir, se evalúa $\text{maxValorRetirado}(W, j+1)$.

Esta relación se expresa formalmente en la recursión:

$$
\text{maxValorRetirado}(W, j) = \max\left\{
\begin{array}{ll}
v_j + \text{maxValorRetirado}(W - w_j, j+1) & \quad \text{(incluir el ítem $ j $)} \\
\text{maxValorRetirado}(W, j+1) & \quad \text{(omitir el ítem $ j $)}
\end{array}
\right.
$$

Además, se establecen casos base para detener la recursión:
- Si $ W = 0 $, la capacidad se ha agotado y no se puede obtener ningún valor adicional, por lo que se retorna 0.
- Si $ W < 0 $, se ha excedido la capacidad de la mochila, y se retorna $-\infty$ (o un valor que indique que esta opción es inviable).
- Si $ j $ excede el número de ítems disponibles, se retorna 0, ya que no hay más decisiones que tomar.

El código inicial implementa esta idea en la función `maxValorRetirado(W, j, pesos, valores)`, en la que se evalúan recursivamente las dos posibilidades para cada ítem.



Una de las dificultades de la formulación recursiva directa es que puede haber cálculos repetidos, ya que el subproblema $\text{maxValorRetirado}(W', j')$ puede evaluarse múltiples veces para los mismos parámetros $W'$ y $j'$. La técnica de **memoización** resuelve este inconveniente almacenando en una tabla los resultados ya computados, evitando recomputaciones y reduciendo de manera drástica el número de llamadas recursivas.

En el código se observa la función `memoizedMaxValorRetirado(W, pesos, valores)` que implementa la técnica de programación dinámica mediante el llenado de una tabla bidimensional $ T[w][j] $ (y otra tabla auxiliar $ S[w][j] $ para la reconstrucción de la solución). Se recorre la tabla en dos bucles anidados:
- El bucle externo recorre todos los valores de $ w $ de 1 a $ W $ en orden ascendente.
- El bucle interno recorre los ítems en orden descendente, desde el último ítem hasta el primero.

De esta forma, para cada combinación de capacidad $ w $ e ítem $ j $, se evalúan las dos posibilidades (incluir u omitir el ítem) y se almacena el mejor resultado. La tabla $ S $ permite recuperar, posteriormente, la lista de ítems que se deben seleccionar para alcanzar el valor óptimo.

**Complejidad pseudopolinómica**

Uno de los aspectos más interesantes del algoritmo de programación dinámica para el problema de la mochila es que su complejidad es **seudopolinómica**. En concreto, el algoritmo tiene una complejidad de $ O(n W) $, donde $n$ es el número de ítems y $W$ es la capacidad máxima de la mochila.

La razón por la que decimos que la complejidad es pseudopolinómica es que $W $ es un valor numérico de entrada y puede ser muy grande en términos de su magnitud, pero su representación en bits es del orden de $\log W$. Por lo tanto, aunque el algoritmo es polinómico en $n$ y $W$, en realidad no es polinómico en el tamaño de la representación binaria de la entrada. Esto significa que el algoritmo de programación dinámica es eficiente cuando $W$ es pequeño o moderado, pero puede volverse ineficiente si $W$ es muy grande, ya que el tiempo de ejecución dependerá linealmente de este valor numérico.

Esta dependencia se debe a que se requiere llenar una tabla con $W+1$ filas, y cada entrada de la tabla se calcula mediante una operación que involucra la iteración sobre los $ n $ ítems. Por tanto, el número total de operaciones es proporcional a $ (W+1) \times n $.

**NP-hard del problema de la mochila**

El problema de la mochila 0-1 es conocido por ser NP-hard, lo que significa que, a menos que $ \text{P} = \text{NP} $, no existe un algoritmo exacto que lo resuelva en tiempo polinómico con respecto al tamaño de la entrada (en particular, respecto a la cantidad de bits necesarios para representar $W$ y los pesos). La dificultad radica en el hecho de que, en el peor caso, la cantidad de posibles subconjuntos de ítems es $2^n$, y un algoritmo ingenuo que evalúe todas las combinaciones tendría una complejidad exponencial.

La programación dinámica consigue un tiempo de $O(nW)$ que es eficiente cuando $W$ es pequeño, pero este algoritmo es pseudopolinómico. Esto es una distinción importante: la eficiencia se mide en función del valor numérico $W$ y no en función de la longitud de la representación binaria de $ W $ (que es aproximadamente $\log W$). Por ello, en casos en que $W$ es grande, la programación dinámica puede resultar impracticable, a pesar de que el número de ítems $n$ sea moderado.

El carácter NP-hard implica que para obtener algoritmos exactos que funcionen en tiempo polinómico en todas las instancias es poco probable que se encuentre una solución, y por ello se recurre a algoritmos aproximados o heurísticos.

Ante la dificultad intrínseca de los problemas NP-hard, se han desarrollado algoritmos de aproximación que pueden encontrar soluciones "cercanas" al óptimo en tiempo polinómico. En el contexto del problema de la mochila, uno de los enfoques más destacados es el **FPTAS** (Fully Polynomial-Time Approximation Scheme).

Un FPTAS es un esquema de aproximación que, para cualquier $\epsilon > 0$, garantiza encontrar una solución cuya calidad es al menos $ (1 - \epsilon) $ de la óptima, y lo hace en tiempo polinómico en $ n $ y $ 1/\epsilon $. La idea principal detrás del FPTAS para la mochila es la siguiente:

- Se realiza una transformación de los valores de los ítems escalándolos y redondeándolos de manera que se reduzca el rango de valores.
- Con estos valores modificados, se aplica la programación dinámica, la cual operará en un espacio reducido.
- La solución obtenida, aunque aproximada, estará dentro de un factor $ (1 - \epsilon) $ del óptimo.


Otra estrategia para abordar el problema de la mochila, en especial cuando se busca una solución exacta, es el uso de **algoritmos de ramificación y poda (branch-bound)**. Este método consiste en explorar de manera sistemática el espacio de soluciones (por ejemplo, el árbol de decisiones de incluir o excluir cada ítem) y utilizar cotas superiores para descartar ramas del árbol que no pueden producir una solución mejor que la ya encontrada.

El proceso de branch- bound se puede resumir en los siguientes pasos

1. **Ramificación**: Se divide el problema en subproblemas más pequeños, generando un árbol de decisiones. Cada nodo del árbol representa una decisión parcial (por ejemplo, haber decidido incluir o excluir ciertos ítems).
2. **Cálculo de cotas superiores**: Para cada nodo, se calcula una cota superior que indica el máximo valor que se podría alcanzar si se completara la solución de manera óptima desde ese nodo.
3. **Poda**: Si la cota superior de un nodo es menor que el valor de la mejor solución encontrada hasta el momento, se descarta (se poda) esa rama, ya que no es necesario explorarla en su totalidad.

Este método puede ser muy eficiente en la práctica para muchas instancias del problema de la mochila, ya que evita explorar partes del espacio de soluciones que no conducirán a mejoras. Sin embargo, en el peor de los casos, la complejidad sigue siendo exponencial.


Debido a la complejidad del problema de la mochila, en particular en sus versiones NP-hard, se han desarrollado **heurísticas** y **metaheurísticas** que permiten encontrar soluciones de buena calidad en tiempos razonables, sin garantizar la optimalidad.


Una heurística clásica para la mochila consiste en utilizar un criterio voraz basado en la relación valor/peso de cada ítem. El procedimiento es el siguiente:

- Se ordenan los ítems en función de la razón $ \frac{v_i}{w_i} $ de forma descendente.
- Se recorre la lista ordenada, añadiendo cada ítem a la mochila siempre que la capacidad restante lo permita.

Esta heurística es muy rápida (con una complejidad de $ O(n \log n) $ debido al ordenamiento) y en muchos casos produce soluciones razonables. Sin embargo, no siempre se alcanza el valor óptimo, especialmente en casos donde la estructura de los ítems no se ajusta bien a este criterio.



Cuando las heurísticas simples no son suficientes o se requiere explorar el espacio de soluciones de manera más exhaustiva, se pueden emplear **metaheurísticas** como:

- **Algoritmos genéticos**: Estos algoritmos imitan el proceso de evolución natural, generando una población de soluciones, evaluando su "aptitud" y utilizando operaciones de cruzamiento y mutación para generar nuevas soluciones. Con el tiempo, la población evoluciona hacia soluciones de mayor calidad.
- **Búsqueda local y recocido simulado (Simulated Annealing)**: Estas técnicas parten de una solución inicial y exploran el vecindario de soluciones posibles, aceptando, en algunos casos, soluciones peores con la esperanza de escapar de óptimos locales y encontrar mejores soluciones globalmente.
- **Optimización por colonia de hormigas o algoritmos de enjambre**: Inspirados en comportamientos naturales, estos algoritmos simulan la interacción de agentes (como hormigas o partículas) que exploran el espacio de soluciones y comparten información para converger hacia buenas soluciones.

Las metaheurísticas son especialmente útiles cuando el problema de la mochila se extiende a versiones más complejas (por ejemplo, la mochila multidimensional o con restricciones adicionales) o cuando el tamaño de la instancia es tan grande que los métodos exactos se vuelven impracticables.



>Un aspecto fundamental al analizar la complejidad del algoritmo de programación dinámica es la forma en que se representa el parámetro $W$ (la capacidad de la mochila). Si $W$ se proporciona en forma de un número entero, la complejidad del algoritmo es $ O(n W) $. Sin embargo, si observamos el tamaño real de la entrada, en términos de bits, $W$ se representa en binario utilizando aproximadamente $\log W$ bits.

>Esta diferencia implica que, aunque el algoritmo es "polinómico" en $W$ (es decir, en la magnitud del número), es **seudopolinómico** respecto al tamaño de la entrada. En otras palabras, el tiempo de ejecución del algoritmo no es polinómico en la longitud de la representación binaria de $ W $, lo cual es una característica clave de muchos problemas NP-hard. Por ello, para instancias en las que $W$ es muy grande, la programación dinámica puede volverse ineficiente, lo que motiva el uso de algoritmos de aproximación (como el FPTAS) o estrategias heurísticas que operen en tiempo polinomial respecto al tamaño en bits de la entrada.


La explicación y el código permiten apreciar cómo un problema NP-hard como la mochila puede abordarse mediante diferentes estrategias. La programación dinámica exacta es una herramienta poderosa cuando el parámetro $ W $ es manejable, ya que se explotan las propiedades de subestructura óptima y superposición de subproblemas. Sin embargo, su complejidad seudopolinómica limita su aplicabilidad a instancias con $ W $ moderado.



Un aspecto central en el análisis de la complejidad del problema de la mochila es comprender la diferencia entre la complejidad en función del valor numérico $ W $ y la complejidad en función del tamaño de la representación binaria de $ W $. En la práctica, cuando se dice que un algoritmo es $ O(n W) $, se asume que $ W $ es una entrada numérica. No obstante, si consideramos que $ W $ se representa en binario, el número de bits necesarios es aproximadamente $\log W$. Esto significa que, en términos de la longitud de la entrada, el algoritmo de programación dinámica tiene una complejidad exponencial, ya que $ W $ puede ser exponencial en $\log W$.

Esta distinción es la razón por la que se clasifica el algoritmo como pseudopolinómico: es eficiente para ciertos rangos de $ W $, pero no se puede considerar un algoritmo polinómico en el sentido clásico (es decir, polinómico en el tamaño total de la representación de la entrada). Este detalle es crucial en el estudio de problemas NP-hard, ya que muchas técnicas de optimización exacta dependen de parámetros numéricos que, aunque pequeños en valor, pueden tener una representación binaria muy compacta.


### Ejercicios 

1. **Trazado de recurrencia y árbol de recursión:**
   - **Ejercicio:** Para un conjunto de ítems con `W = 20`, `pesos = [1, 5, 20, 35, 90]` y `valores = [15, 14.5, 19.2, 19.8, 195.2]`, traza el árbol de llamadas recursivas que realiza la función `maxValorRetirado`.  Identifica las subestructuras óptimas y los casos base, y entender cómo se exploran las posibles combinaciones.
   
2. **Análisis de complejidad:**
   - **Ejercicio:** Analiza la complejidad temporal de la versión recursiva pura de `maxValorRetirado` y compárala con la versión memoizada (`memoizedMaxValorRetirado`).  Discute el efecto de la memoización sobre la reducción del número de cálculos repetidos y explicar por qué la versión recursiva tiene una complejidad exponencial.

3. **Recuperación de la solución:**
   - **Ejercicio:** Modifica la función `memoizedMaxValorRetirado` para que, además de imprimir los ítems seleccionados, retorne una lista de tuplas con (ítem, peso, valor).  

4. **Validación de resultados:**
   - **Ejercicio:** Ejecuta `memoizedMaxValorRetirado` con diferentes valores de `W` (por ejemplo, 50, 100 y 200) y verifica manualmente (o con cálculos auxiliares) que la solución encontrada respeta el límite de peso y maximiza el valor.  Comprende la robustez del algoritmo ante distintos escenarios y cómo cambia la selección de ítems.

5. **Trazado y análisis de la recurrencia:**
   - **Ejercicio:** Para `W = 25` con los mismos vectores de `pesos` y `valores`, traza la recurrencia de la función `maxRemovido` y discute qué combinaciones de ítems llevan al valor óptimo.  Identifica cómo se resuelve el problema cuando se pueden usar múltiples unidades de cada ítem.

6. **Memoización y recuperación de la solución:**
   - **Ejercicio:** La función `memo_maxRemovido` recupera la solución en forma de lista de mensajes. Modifica la función para que retorne un vector de cantidades `[n_1, n_2, ..., n_k]`, donde cada `n_i` indica cuántas veces se seleccionó el ítem i.  Profundiza en la recuperación de la solución en problemas de programación dinámica y practicar el manejo de estructuras de datos.

7. **Comparación de estrategias:**
   - **Ejercicio:** Discute las diferencias entre la solución para la mochila 0-1 y la solución para la mochila con número ilimitado de ítems. ¿En qué escenarios del mundo real podría ser más adecuado cada modelo?. Desarrolla una visión crítica sobre la aplicabilidad de cada enfoque y entender la diferencia en la restricción del número de ítems.

8. **Optimización del algoritmo:**
   - **Ejercicio:** Investiga cómo se puede reducir el espacio de memoria en la versión de la mochila con número ilimitado de ítems. ¿Es posible utilizar una única lista unidimensional en lugar de dos listas (T y S)? Implementa la versión optimizada y compárala en términos de rendimiento.  
9. **Extensión del problema:**
   - **Ejercicio:** Considera el caso en el que algunos ítems tengan restricciones adicionales (por ejemplo, ciertos ítems solo pueden tomarse si se toma otro ítem). Propón cómo modificar la recurrencia y la memoización para incorporar estas restricciones adicionales.  

10. **Análisis comparativo:**
    - **Ejercicio:** Escribe un informe breve comparando las técnicas de fuerza bruta, recursión pura, memoización y programación dinámica en el contexto de estos dos problemas. ¿Cuáles son las ventajas y desventajas de cada enfoque?  

In [ ]:
## Tus respuestas